In [1]:
import xilMisPython as xmis
import os
import argparse
import struct
import time

In [2]:
def readBin(filename, readSize):
    with open(filename, 'rb') as fh:
        fileSize = os.path.getsize(filename)

        if 0 < readSize != fileSize:
            print(f"WARNING: file {filename} size {fileSize} doesn't match required size {readSize}")

        assert (fileSize >= readSize)

        vec = []
        for i in range(int(fileSize/4)):
            vec.append(struct.unpack('<i', fh.read(4))[0])  # read bytes as little-endian 4 byte integers

    return vec

In [3]:
xclbin_path = os.environ.get('XCLBIN_FILE')
deviceNames = os.environ.get('DEV_NAME')
in_dir = os.environ.get('DATA_DIR')

In [4]:
# search for input file matrix meta data information stored in infos.txt
with open(in_dir + "/infos.txt") as fh:
    fh.readline()
    n = int(fh.readline())
    fh.readline()
    nz = int(fh.readline())

In [5]:
# set options for MIS
opt = xmis.options()
opt.xclbinPath = xmis.xString(xclbin_path)
opt.deviceNames = xmis.xString(deviceNames)

In [6]:
# create MIS object
mis = xmis.MIS(opt)

In [7]:
# create CSR arrays and graph object
h_rowPtr = readBin(in_dir + "/rowPtr.bin", (n + 1) * 4)  # multiplier of 4 for integer data type
h_colIdx = readBin(in_dir + "/colIdx.bin", nz * 4)       # multiplier of 4 for integer data type
graph = xmis.GraphCSR(h_rowPtr, h_colIdx)

In [8]:
# initialize MIS object with the graph
mis.setGraph(graph)

In [9]:
# initialize the FPGA device for MIS run
mis.startMis()

In [12]:
# execute MIS on FPGA
start = time.perf_counter()
mis_vertex_list = mis.executeMIS()
end = time.perf_counter()
elapsed = end-start

In [13]:
# get size of the MIS
vertex_count = mis.count()  # can also do len(mis_vertex_list)

print(f"\nFound MIS with {vertex_count} vertices within {elapsed:0.6f} sec")


Found MIS with 25 vertices within 0.000911 sec
